# Case Ilumeo

A **Marca 01** entrou em contato com a Ilumeo e contratou a Delfos, nosso produto de **tracking de imagem de marca**, com objetivo de **avaliar sua performance no mercado e avaliar o posicionamento da sua marca em comparação aos concorrentes**. Com esse objetivo devemos seguir alguns passos para montar a apresentação: 

1. **Limpeza de dados** - Necessário verificar se seguiremos com a base de dados da forma que recebemos do time de coleta de dados. Existe alguma resposta que devemos desconsiderar? A qualidade de respostas está adequada para uma anáise?
2. **Big numbers e descritivas** - Antes de elaborar uma apresentação ao cliente, com recomendações e análises é necessário criar um documento (Power BI) para trazer todas as descritivas que a base trás. Separamos as perguntas em alguns temas: Perfil demográfico, hábitos com a categoria, funil de marca e avaliação de marca. Use para auxiliar na criação das descritivas.
3. **Apresentação** - Analisar os dados e criar uma apresentação. 

In [396]:
import pandas as pd 
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [397]:
dados = pd.read_excel('dados.xlsx')

dados.head(2)

,Q01respondent_id,Q03date_created,Q06semanadoano,Q09semanadoprojeto,Q11Profissao,Q13Estado,Q14Sexo,Q15Idade,Q16Escolaridade,Q17Renda,Q18PossuiSeguroNenhuma das anteriores,Q18PossuiSeguroSeguro de vida,Q18PossuiSeguroSeguro de automóvel,Q18PossuiSeguroSeguro residencial,Q18PossuiSeguroSeguro de saúde,Q18PossuiSeguroSeguro de bens duráveis (ex: eletrônicos),Q18PossuiSeguroOutros,Q19ContratouSeguroSeguro de vida,Q19ContratouSeguroSeguro de automóvel,Q19ContratouSeguroSeguro residencial,Q19ContratouSeguroSeguro de saúde,Q19ContratouSeguroSeguro de bens duráveis (ex: eletrônicos),Q19ContratouSeguroOutros,Q20PretendeRenovar,Q21PretendeContratar,Q22ContratarSeguroNenhuma das anteriores,Q22ContratarSeguroSeguro de vida,Q22ContratarSeguroSeguro de automóvel,Q22ContratarSeguroSeguro residencial,Q22ContratarSeguroSeguro de saúde,Q22ContratarSeguroSeguro de bens duráveis (ex: eletrônicos),Q22ContratarSeguroOutros,Q23AtitudeGeralSeguroImportancia,Q23AtitudeGeralSeguroPrioridadeOrcamento,Q23AtitudeGeralSeguroSubstituiEntretenimento,Q24OrdenarSeguroSeguro de vida,Q24OrdenarSeguroSeguro de automóvel,Q24OrdenarSeguroSeguro residencial,Q24OrdenarSeguroSeguro de saúde,Q24OrdenarSeguroSeguro de bens duráveis (ex: eletrônicos),Q25UsoInternet1,Q25UsoInternet2,Q25UsoInternet3,Q25UsoInternet4,Q26Transacoes1,Q26Transacoes2,Q26Transacoes3,Q26Transacoes4,Q26Transacoes5,Q26Transacoes6,Q27Antenado1,Q27Antenado2,Q27Antenado3,Q27Antenado4,Q31AWNenhuma das anteriores,Q31AWMarca01,Q31AWMarca02,Q31AWMarca03,Q31AWMarca04,Q31AWMarca05,Q31AWMarca06,Q31AWMarca07,Q32KNNenhuma das anteriores,Q32KNMarca01,Q32KNMarca02,Q32KNMarca03,Q32KNMarca04,Q32KNMarca05,Q32KNMarca06,Q32KNMarca07,Q33LINenhuma das anteriores,Q33LIMarca01,Q33LIMarca02,Q33LIMarca03,Q33LIMarca04,Q33LIMarca05,Q33LIMarca06,Q33LIMarca07,Q34CONenhuma das anteriores,Q34COMarca01,Q34COMarca02,Q34COMarca03,Q34COMarca04,Q34COMarca05,Q34COMarca06,Q34COMarca07,Q35Preferencia,Q36CLNenhuma das anteriores,Q36CLMarca01,Q36CLMarca02,Q36CLMarca03,Q36CLMarca04,Q36CLMarca05,Q36CLMarca06,Q36CLMarca07,Q37ContratouPrimeiraVez,Q39ContratouPrimeiraVezSeguro de vida,Q39ContratouPrimeiraVezSeguro de automóvel,Q39ContratouPrimeiraVezSeguro residencial,Q39ContratouPrimeiraVezSeguro de saúde,Q39ContratouPrimeiraVezSeguro de bens duráveis (ex: eletrônicos),Q39ContratouPrimeiraVezOutros,Q40AvaliouMarca01,Q41AvaliouMarca02,Q42AvaliouMarca03,Q43AvaliouMarca04,Q44AvaliouMarca05,Q45AvaliouMarca06,Q46AvaliouMarca07,Q47Credibilidade1,Q47Credibilidade2,Q47Credibilidade3,Q48Diversao1,Q48Diversao2,Q48Diversao3,Q49Audacia1,Q49Audacia2,Q49Audacia3,Q50Sofisticacao1,Q50Sofisticacao2,Q50Sofisticacao3,Q51Sensibilidade1,Q51Sensibilidade2,Q51Sensibilidade3,Q52AltaClasse1,Q52AltaClasse2,Q52AltaClasse3,Q52AltaClasse4,Q52AltaClasse5,Q53Acessibilidade1,Q53Acessibilidade2,Q53Acessibilidade3,Q54Lideranca1,Q54Lideranca2,Q54Lideranca3,Q54Lideranca4,Q55Descolada1,Q55Descolada2,Q55Descolada3,Q56Variedade1,Q56Variedade2,Q56Variedade3,Q56Variedade4,Q56Variedade5,Q57ServicoHumanizado1,Q57ServicoHumanizado2,Q57ServicoHumanizado3,Q57ServicoHumanizado4,Q57ServicoHumanizado5,Q57ServicoHumanizado6,Q58Qualidade1,Q58Qualidade2,Q58Qualidade3,Q58Qualidade4,Q59Atratividade1,Q59Atratividade2,Q60ResponsabilidadeSocioAmbiental1,Q60ResponsabilidadeSocioAmbiental2,Q60ResponsabilidadeSocioAmbiental3,Q61Autenticidade1,Q61Autenticidade2,Q61Autenticidade3,Q61Autenticidade4,Q61Autenticidade5,Q61Autenticidade6,Q61Autenticidade7,Q61Tecnologica,Q62Beneficios1,Q62Beneficios2,Q62Beneficios3,Q63Dinamismo1,Q63Dinamismo2,Q63Dinamismo3,Q64CuidadoHumano1,Q64CuidadoHumano2,Q64CuidadoHumano3,Q65QualidadeServico1,Q65QualidadeServico2,Q65QualidadeServico3,Q66CustoBeneficio1,Q66CustoBeneficio2,Q66CustoBeneficio3,Q67Diversidade1,Q68Risco1,Q68Risco2,Q68Risco3,Q69Satisfacao1,Q69Satisfacao2,Q69Satisfacao3,Q69Satisfacao4,Q70Lealdade1,Q70Lealdade2,Q70Lealdade3,Q71IntencaoCompra1,Q71IntencaoCompra2,Q71IntencaoCompra3,Q72Recomendacao1,Q72Recomendacao2,Q72Recomendacao3,Q73BrandEquity1,Q73BrandEquity2,Q73

In [398]:
print(f"O dataframe possui {dados.shape[0]} linhas e {dados.shape[1]} colunas.\n")

print('Valores duplicados:', dados.duplicated().any().sum())
print('Valores nulos:', dados.isna().values.sum())

O dataframe possui 511 linhas e 210 colunas.

Valores duplicados: 0
Valores nulos: 32365


In [399]:
dados.dropna(axis='columns', how='all', inplace=True)

dropar = [col for col in dados.columns if 'Nenhuma das anteriores' in col]
dados = dados.drop(columns=dropar)

dados = dados.drop(columns=dados[['Q06semanadoano', 'Q09semanadoprojeto']])

dados['Marca'] = dados['Q35Preferencia']

In [400]:
regioes = {
    "Norte": ["Acre", "Amapá", "Amazonas", "Pará", "Rondônia", "Roraima", "Tocantins"],
    "Nordeste": ["Alagoas", "Bahia", "Ceará", "Maranhão", "Paraíba", "Pernambuco", "Piauí", "Rio Grande do Norte", "Sergipe"],
    "Centro-Oeste": ["Distrito Federal", "Goiás", "Mato Grosso", "Mato Grosso do Sul"],
    "Sudeste": ["Espírito Santo", "Minas Gerais", "Rio de Janeiro", "São Paulo"],
    "Sul": ["Paraná", "Rio Grande do Sul", "Santa Catarina"]
}

def obter_regiao(estado):
    for regiao, estados_regiao in regioes.items():
        if estado in estados_regiao:
            return regiao

dados['Região'] = dados['Q13Estado'].apply(obter_regiao)

dados[['Q13Estado', 'Região']]

,Q13Estado,Região
0,São Paulo,Sudeste
1,Minas Gerais,Sudeste
2,Rio de Janeiro,Sudeste
3,Santa Catarina,Sul
4,Rondônia,Norte
...,...,...
506,Minas Gerais,Sudeste
507,Rio de Janeiro,Sudeste
508,São Paulo,Sudeste
509,Pernambuco,Nordeste


In [401]:
dados['Q15Idade'] = dados['Q15Idade'].replace('[^0-9]','', regex=True).astype('Int16')

faixas = {
    '18 anos ou mais': (18, 24),
    '25 anos ou mais': (25, 39),
    '40 anos ou mais': (40, 59),
    '60 anos ou mais': (60, 100)
}

def cat_idade(idade):
    for faixa, (inicio, fim) in faixas.items():
        if inicio <= idade <= fim:
            return faixa

dados['FaixaEtaria'] = dados['Q15Idade'].astype(int)

dados['FaixaEtaria'] = dados['Q15Idade'].apply(cat_idade) 
dados['FaixaEtaria'].unique()

array(['40 anos ou mais', '18 anos ou mais', '25 anos ou mais',
       '60 anos ou mais'], dtype=object)

In [402]:
dados['Q16Escolaridade'] = dados['Q16Escolaridade'].str.split(' \(').str[0]
dados['Q16Escolaridade'].unique()

array(['Ensino superior', 'Ensino médio', 'Pós-graduação', 'Mestrado',
       'Doutorado', 'Ensino fundamental'], dtype=object)

In [403]:
# Defina as faixas de valores para cada classe
classes = {
    "Classes AB": [
        'Entre R$ 7.053,00 e R$ 15.071,00',
        'Entre R$ 15.071,00 e R$ 20.888,00',
        'Acima de R$ 20.888,00',
        'Entre R$ 16.368,00 e R$ 22.865,00',
        'Acima de R$ 22.866,00',
        'Entre R$ 7.420,00 e R$ 16.367,00'
    ],
    "Classe C": [
        'Entre R$ 3.932,00 e R$ 7.419,00',
        'Entre R$ 1.196,50 e R$ 2.165,00',
        'Entre R$ 2.239,00 e R$ 3.931,00',
        'Entre R$ 3.778,50 e R$ 7.053,00',
        'Entre R$ 2.165,00 e R$ 3.778,50',
        'Entre R$ 1.216 e R$ 2.238,00'
    ],
    "Classe DE": [
        'Até R$ 780,00',
        'Entre R$ 768,00 e R$ 1.196,50',
        'Entre R$ 781,00 e R$ 1.215,00'
    ]
}

# Defina uma função para verificar a classe de cada valor
def criando_classes(value):
    for classe, faixas in classes.items():
        for faixa in faixas:
            if value == faixa:
                return classe

# Aplicar a função para verificar a classe de cada valor
dados['Classe'] = dados['Q17Renda'].apply(criando_classes) 
dados[['Q17Renda', 'Classe']]

,Q17Renda,Classe
0,"Entre R$ 3.932,00 e R$ 7.419,00",Classe C
1,"Entre R$ 1.196,50 e R$ 2.165,00",Classe C
2,"Entre R$ 1.196,50 e R$ 2.165,00",Classe C
3,"Entre R$ 3.932,00 e R$ 7.419,00",Classe C
4,"Entre R$ 2.239,00 e R$ 3.931,00",Classe C
...,...,...
506,"Entre R$ 7.420,00 e R$ 16.367,00",Classes AB
507,"Entre R$ 781,00 e R$ 1.215,00",Classe DE
508,"Entre R$ 3.932,00 e R$ 7.419,00",Classe C
509,"Entre R$ 3.932,00 e R$ 7.419,00",Classe C


In [404]:
novos_nomes = {}
for coluna in dados.columns[99:200].append(dados.columns[35:49]): 
    novo_nome = coluna[3:] 
    novos_nomes[coluna] = novo_nome

dados.rename(columns=novos_nomes, inplace=True)

dados.columns[99:200], dados.columns[35:49]

(Index(['Credibilidade1', 'Credibilidade2', 'Credibilidade3', 'Diversao1',
        'Diversao2', 'Diversao3', 'Audacia1', 'Audacia2', 'Audacia3',
        'Sofisticacao1',
        ...
        'BrandEquity2', 'BrandEquity3', 'BrandEquity4', 'BrandEquity5',
        'PagarMais1', 'PagarMais2', 'PagarMais3', 'Identificacao1',
        'Identificacao2', 'Identificacao3'],
       dtype='object', length=101),
 Index(['UsoInternet1', 'UsoInternet2', 'UsoInternet3', 'UsoInternet4',
        'Transacoes1', 'Transacoes2', 'Transacoes3', 'Transacoes4',
        'Transacoes5', 'Transacoes6', 'Antenado1', 'Antenado2', 'Antenado3',
        'Antenado4'],
       dtype='object'))

In [405]:
novos_nomes = {}
for coluna in dados.columns[92:99]: 
    novo_nome = coluna[10:] 
    novos_nomes[coluna] = novo_nome

dados.rename(columns=novos_nomes, inplace=True)
dados.columns[92:99]

Index(['Marca01', 'Marca02', 'Marca03', 'Marca04', 'Marca05', 'Marca06',
       'Marca07'],
      dtype='object')

In [406]:
dados.replace('[Insert text from Other]','Outro',inplace=True)
dados.replace('Outro (especifique)','Outro', inplace=True)

In [407]:
selecionadas = dados.iloc[:, 99:200]

colunas_tratadas = selecionadas.columns.str.extract(r'(\D+)')[0]

medias_por_grupo = {}

for nome_coluna in colunas_tratadas.unique():
    colunas_grupo = selecionadas.filter(like=nome_coluna)
    media_grupo = colunas_grupo.mean(axis=1, skipna=True)
    media_grupo_dividida = media_grupo / 7  # Dividindo pela quantidade de dias na semana
    medias_por_grupo[nome_coluna] = media_grupo_dividida

media_grupo = pd.DataFrame(medias_por_grupo).T

media_grupo = media_grupo.mean(axis=1)

# Criando um novo DataFrame com a média total e renomeando a coluna
media_grupo = pd.DataFrame({'Media_Indices': media_grupo})

media_grupo.reset_index(inplace=True)

media_grupo.rename(columns={'index': 'Indices'}, inplace=True)

media_grupo.head()

,Indices,Media_Indices
0,Credibilidade,1.146398
1,Diversao,0.984810
2,Audacia,1.081074
3,Sofisticacao,1.006057
4,Sensibilidade,0.838785


In [408]:
selecionadas = dados.iloc[:, 99:200]

indices = selecionadas.mean().reset_index()

indices.columns = ['Indices', 'Média']

indices.head()

,Indices,Média
0,Credibilidade1,8.041096
1,Credibilidade2,7.984344
2,Credibilidade3,8.048924
3,Diversao1,6.884540
4,Diversao2,6.739726


In [409]:
dados.to_excel('dados_cat.xlsx', index=False, sheet_name='Base')

with pd.ExcelWriter('dados_cat.xlsx', mode='a') as writer:
    indices.to_excel(writer, sheet_name='Media_Marca',index=False)
    media_grupo.to_excel(writer, sheet_name='Media_Grupo',index=False)

### Referências
Rodrigues, L. (2023, 26 de agosto). Relação com crédito bancário varia conforme classe social, diz estudo. Agência Brasil. Acesse [aqui](https://agenciabrasil.ebc.com.br/economia/noticia/2023-08/relacao-com-credito-bancario-varia-conforme-classe-social-diz-estudo).

Instituto Brasileiro de Geografia e Estatística (IBGE). (s.d.). Educação. Acesse [aqui](https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/18317-educacao.html).